### Start SparkContext

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

### imports

In [62]:
import os
import datetime as dt
import pytz # python timezones

import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
matplotlib.rcParams['figure.figsize'] = (16, 20)

from pyspark.sql import SparkSession
import pyspark.sql.functions as func # resuse as func.coalace for example
from pyspark.sql.types import StringType, IntegerType, FloatType, DoubleType, DecimalType
from pyspark import SparkFiles

import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame # loading boundaries data
from shapely.geometry import Point, Polygon, shape # creating geospatial data
from shapely import wkb, wkt # creating and parsing geospatial data

### abbreviation
**df** = Pandas DataFrame \
**gdf** = Geopandas GeoDataFrame \
**sdf** = Spark DataFrame

### GPS signals

Python - converting UTC millisecond timestamp to local time \
https://stackoverflow.com/questions/52451105/python-converting-utc-millisecond-timestamp-to-local-time

In [63]:
def date_from_utc_ms_ts(utc_ms_ts, tz_info) -> str:
    """Return a date (yyyy-mm-dd) from a string of utc timestamp in milliseconds and a timezone.

    :param utc_ms_ts: Unix UTC timestamp in milliseconds (int or str)
    :param tz_info: timezone info (pytz.tzfile)
    :return: date yyyy-mm-dd (str)
    """
    # convert from time stamp to datetime
    utc_datetime = dt.datetime.utcfromtimestamp(int(utc_ms_ts) / 1000)
    # set the timezone to UTC, and then convert to desired timezone
    date = utc_datetime.replace(tzinfo=pytz.timezone('UTC')).astimezone(tz_info).strftime('%Y-%m-%d')
    return date

In [64]:
timestamp = "1609459727000"

In [65]:
date = date_from_utc_ms_ts(timestamp, pytz.timezone('Europe/Berlin'))
print(type(date))
date

<class 'str'>


'2021-01-01'

In [66]:
timestamp = 1609459727000

In [67]:
date = date_from_utc_ms_ts(timestamp, pytz.timezone('Europe/Berlin'))
print(type(date))
date

<class 'str'>


'2021-01-01'